# get data body in spark

In [1]:
data = sc.wholeTextFiles('data/reut2-02*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                   .map(lambda x:x.replace('\n', ' '))

newsArticles.take(1)

['If the dollar goes the way of Wall Street, Japanese will finally move out of dollar investments in a serious way, Japan investment managers say.     The Japanese, the dominant foreign investors in U.S. Dollar securities, have already sold U.S. Equities.     But "if the dollar falls steeply, which did not happen yesterday, Japanese investors will definitely try to withdraw significant funds from U.S. Shares," said Akira Kawakami, deputy manager of Nomura Investment Trust and Management Co Ltd\'s international investment department.     An unstable, lower dollar would also affect Japanese investment in U.S. Bonds. "Japan-U.S. Interest rate differentials, which currently look wide enough, mean nothing in the absence of dollar stability," said Kawakami.     U.S. Bonds could benefit due to a gloomy economic picture following the estimated huge losses in stocks by major U.S. Institutional and individual investors, he said. The effect should be to rule out any U.S. Interest rate rise.     B

# (1) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [2]:
k = 5
shingles = newsArticles.flatMap(lambda x:[x[i:i+k] for i in range(len(x)-k+1)]).distinct()
newsArticlesBC = sc.broadcast(newsArticles.collect())
kShinglesMatrix = shingles.map(lambda s:[1 if s in a else 0 for a in newsArticlesBC.value]).collect()

In [3]:
print('Shingles=', len(kShinglesMatrix))
print('Articles=', len(kShinglesMatrix[0]))

Shingles= 13788
Articles= 1237


In [4]:
with open('result/kShinglesMatrix.txt', 'w') as result:
    for row in kShinglesMatrix:
        result.write(str(row))

# (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [8]:
h = 10

import random

minHashSignatures = list()
articles_count = newsArticles.count()
for i in range(h):
    rndIdx = random.sample(range(articles_count), articles_count)
    minHash = [-1]*articles_count
    while -1 in minHash:
        row = rndIdx.pop()
        for j in range(len(minHash)):
            if (minHash[j] == -1 or minHash[j] > row) and kShinglesMatrix[row][j] == 1:
                minHash[j] = row
    minHashSignatures.append(minHash)

In [9]:
with open('result/minHashSignatures.txt', 'w') as result:
    for row in minHashSignatures:
        result.write(str(row))